In [1]:
%matplotlib ipympl

from scipy.spatial.distance import pdist, squareform
from scipy.sparse.linalg import eigs, eigsh
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# File Paths
data_path = "./data"
figure_path = "./plots"
abundance_table_path = f"{data_path}/abundance_table_97.shared"
metadata_path = f"{data_path}/SuperTransect_mapping_file.csv"

In [3]:
# Abundance Table
with open(abundance_table_path, "r") as file_literal:
    raw_abundance_data = [line.strip().split("\t") for line in file_literal]
    otu_names = raw_abundance_data[0][3:]
    sample_names = list(map(int, [line[1] for line in raw_abundance_data[1:]]))
    otu_counts = [line[3:] for line in raw_abundance_data[1:]]
abundance_table = pd.DataFrame(
    np.array(otu_counts, dtype=np.int64),
    index=sample_names,
    columns=otu_names)
abundance_table["Abundance"] = abundance_table.sum(axis=1)
abundance_table["Presence"] = abundance_table.drop("Abundance", axis=1).where(
    abundance_table == 0, 1).sum(axis=1)

# Metadata
metadata = pd.read_csv(metadata_path, index_col=0)

In [57]:
# Filtering Method
mosquito_metadata = metadata.loc[metadata["sample_type"] == "Mosquito"]
mosquito_abundance = abundance_table.filter(
    items=list(mosquito_metadata.index), axis=0).drop(["Abundance", "Presence"],axis=1).drop([105279, 105525, 105502, 105312, 105546], axis=0)
mosquito_metadata = mosquito_metadata.loc[mosquito_abundance.index]

In [60]:
# Filtering Method
mosquito_metadata = metadata.loc[metadata["host"] == "Animal"]
mosquito_abundance = abundance_table.filter(
    items=list(mosquito_metadata.index), axis=0).drop(["Abundance", "Presence"],axis=1)
mosquito_metadata = mosquito_metadata.loc[mosquito_abundance.index]

In [55]:
def abundance_to_eigenvector(filtered_abundance_table, debug=False):
    adjacency_matrix = squareform(pdist(filtered_abundance_table, metric="minkowski", p=1))
    kernel = np.exp(- (adjacency_matrix ** 2) / (3000**2))
    diagonal = np.diag(np.sum(kernel,axis=1))
    laplacian = diagonal - kernel
    eigenvalues, eigenvectors = eigs(laplacian, k=len(laplacian) - 1, M=diagonal)
    sample_eigens = zip(eigenvalues.real, eigenvectors.T, filtered_abundance_table.index)
    eigenvalues, eigenvectors, sample_ids = zip(*sorted(sample_eigens, key = lambda tup:tup[0]))

    if debug:
        print("Adjacency Matrix:\n", adjacency_matrix, "\n")
        print("Kernel:\n", kernel, "\n")
        print("Diagonal:\n", diagonal, "\n")
        print("Laplacian:\n", laplacian, "\n")
        print("Eigenvalues:\n", eigenvalues, "\n")
        print("Eigenvectors:\n", eigenvectors, "\n")
        print("Sample ID's:\n", sample_ids, "\n")
        
    return eigenvectors, filtered_abundance_table.index

def eigenvector_to_plot(eigenvectors, text, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.scatter3D(eigenvectors[1], eigenvectors[2], eigenvectors[3])
    #for label, x, y, z in zip(text, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
    #    ax.text(x, y, z, label, None)
    plt.title(title)
    plt.show()

In [56]:
eigenvector_to_plot(*abundance_to_eigenvector(mosquito_abundance), "Mosquito Generalized Eigenvalue Dropped None")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
eigenvector_to_plot(*abundance_to_eigenvector(mosquito_abundance), "Mosquito Generalized Eigenvalue Dropped 3")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
eigenvector_to_plot(*abundance_to_eigenvector(mosquito_abundance), "Mosquito Generalized Eigenvalue Dropped 5")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
eigenvector_to_plot(*abundance_to_eigenvector(mosquito_abundance), "Animal Generalized Eigenvalue")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …